In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop 
from keras.utils.data_utils import get_file
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, TensorBoard


import tensorflow as tf
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
!wget -qq https://www.dropbox.com/s/szpkcod17qqxxbq/rick_and_morty.txt?dl=0

with io.open('rick_and_morty.txt?dl=0', encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 1379170


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
#includes new line return character

#have to convert letters to math, i.e. assign letter to a number

total chars: 74


In [4]:
maxlen = 100
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 459690


***Testing***

In [5]:
# moving frame, in characters of 3

print(sentences[100])
print(sentences[101])
print(sentences[102])

: (spills alcohol on morty's bed) come on, i got a surprise for you. (drags morty by the ankle) come
spills alcohol on morty's bed) come on, i got a surprise for you. (drags morty by the ankle) come on
lls alcohol on morty's bed) come on, i got a surprise for you. (drags morty by the ankle) come on, h


In [0]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [7]:
x.shape

(459690, 100, 74)

In [8]:
 maxlen, len(chars)

(100, 74)

In [9]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               338944    
_________________________________________________________________
dense (Dense)                (None, 74)                19018     
_________________________________________________________________
activation (Activation)      (None, 74)                0         
Total params: 357,962
Trainable params: 357,962
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds, temperature=0.6):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

**Training Model**

In [0]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('rick: i got a surprise for you, morty.' + sentence)
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [0]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

reduce_LR = ReduceLROnPlateau(monitor='loss',factor = 0.9, patience=1,cooldown=0, min_lr = 0.00001)

**Generating Dialogue**

In [15]:
model.fit(x, y,
          batch_size=256,
          epochs=20,
          callbacks=[print_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
459520/459690 [============================>.] - ETA: 0s - loss: 2.5967
----- Generating text after Epoch: 0
----- diversity: 0.2
rick: i got a surprise for you, morty.e. rick, morty, and hemorrhage all sit on a couch, watching television. hemorrhage is dressed in a "
e. rick, morty, and hemorrhage all sit on a couch, watching television. hemorrhage is dressed in a "inger and pulls out the stard of the stard to the stard to the stard and and and it s the wall and stuff the member and morty stare to the stard and and and it s and in the portal of the stard and and and the stard to the smands and morty and morty stards the stop the stup of the cardet to the stard and pulls and morty. 
rick: i m summer stare and the stard and and morty stares of the stard and st
----- diversity: 0.5
rick: i got a surprise for you, morty.e. rick, morty, and hemorrhage all sit on a couch, watching television. hemorrhage is dressed in a "
e. rick, mo

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 un't goed and just put remenvele to your look space in ibvenalac your offected armoss. my harts st
----- diversity: 1.2
rick: i got a surprise for you, morty. the device. 
morty: i think it s cool. 
rick: (angry) science is cool! magic is for the moment. for
 the device. 
morty: i think it s cool. 
rick: (angry) science is cool! magic is for the moment. for free]
a! spung. farts out orear forme! 
[int. but master  dramat. 
rather , ? summer, tretty pasthe of rick? 
beth: oh, god, what s beening? i'm "good. , summer. he feels a lrock onumy. he apautoole jobs beth, tuniony, the horset.) dows them.. 
 darium: okay, beaind can't creahe? 
summer: about floors, try holicodrats humary, jeres louldyker, 
kingings made maina mess zoons.[huggle"] 
an onity: c
459690/459690 [==============================] - 397s 865us/sample - loss: 1.3070
Epoch 7/20
459520/459690 [============================>.] - ETA: 0s - loss: 1.2917
----- Generating text after Epoch: 6
----- diversity: 0.2
rick: i got a su